In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import Adam

In [4]:
np.set_printoptions(suppress=True)

# SINGLE ITEM

## Generate data

In [5]:
# number of obs
n = 1000

In [6]:
# random prices and other variables
prices = np.random.uniform(20, 200, size=(n, 1)).astype(np.float32)
xs = np.random.rand(n,5).astype(np.float32)

In [7]:
# create beta coefficients
beta_0 = np.array(([300],))
beta_p = np.array(([-1.5],))
betas_x = np.random.normal(size=(5,1)).astype(np.float32)

In [8]:
# optimal price
opt_price = -beta_0/(2*beta_p)
opt_price

array([[100.]])

In [9]:
# noise
noise = np.random.normal(size=(n,1)).astype(np.float32)

In [10]:
# create demand to forecast
y = beta_0 + beta_p*prices + np.dot(xs, betas_x) + noise
y = np.maximum(0, y)

In [11]:
# plt.hist(y)

In [12]:
# train/test ids
trn = np.arange(int(n*.8))
tst = np.arange(int(n*.8), n)

## Model demand

In [13]:
# price and X inputs
inp_p = Input((1,), name='inp_p')
inp_x = Input((5,), name='inp_x')

In [14]:
# concat the inputs
x = Concatenate(name='concat')([inp_p, inp_x])

# single dense hidden layer
x = Dense(3, name='hid')(x)

# output for predicting demand
out = Dense(1, name='out')(x)

# build and compile the model
model = Model([inp_p, inp_x], out)
model.compile(Adam(lr=.01), loss='mean_squared_error')

In [15]:
# train
hist = model.fit([prices[trn], xs[trn]], y[trn],
                 validation_data=[[prices[tst], xs[tst]], y[tst]],
                 epochs=200)

Train on 800 samples, validate on 200 samples
Epoch 1/200
800/800 [==============================] - 0s 120us/step - loss: 17755.0689 - val_loss: 17705.4463
Epoch 2/200
800/800 [==============================] - 0s 67us/step - loss: 15616.2720 - val_loss: 17573.2849
Epoch 3/200
800/800 [==============================] - 0s 67us/step - loss: 15423.6966 - val_loss: 17385.5369
Epoch 4/200
800/800 [==============================] - 0s 60us/step - loss: 15266.8336 - val_loss: 17196.5168
Epoch 5/200
800/800 [==============================] - 0s 44us/step - loss: 15091.8268 - val_loss: 16993.8998
Epoch 6/200
800/800 [==============================] - 0s 52us/step - loss: 14894.8808 - val_loss: 16729.5309
Epoch 7/200
800/800 [==============================] - 0s 67us/step - loss: 14633.0711 - val_loss: 16424.4441
Epoch 8/200
800/800 [==============================] - 0s 50us/step - loss: 14333.3721 - val_loss: 16043.8994
Epoch 9/200
800/800 [==============================] - 0s 70us/step - los

800/800 [==============================] - 0s 62us/step - loss: 868.5398 - val_loss: 688.3082
Epoch 76/200
800/800 [==============================] - 0s 67us/step - loss: 857.2435 - val_loss: 792.1251
Epoch 77/200
800/800 [==============================] - 0s 64us/step - loss: 804.5646 - val_loss: 647.8252
Epoch 78/200
800/800 [==============================] - 0s 63us/step - loss: 777.0364 - val_loss: 632.2485
Epoch 79/200
800/800 [==============================] - 0s 60us/step - loss: 745.4908 - val_loss: 592.4602
Epoch 80/200
800/800 [==============================] - 0s 64us/step - loss: 713.6809 - val_loss: 647.8639
Epoch 81/200
800/800 [==============================] - 0s 62us/step - loss: 700.7122 - val_loss: 570.5094
Epoch 82/200
800/800 [==============================] - 0s 62us/step - loss: 676.0904 - val_loss: 535.6251
Epoch 83/200
800/800 [==============================] - 0s 61us/step - loss: 640.5685 - val_loss: 574.6256
Epoch 84/200
800/800 [============================

800/800 [==============================] - 0s 61us/step - loss: 1.6316 - val_loss: 1.9904
Epoch 153/200
800/800 [==============================] - 0s 60us/step - loss: 1.4308 - val_loss: 1.3881
Epoch 154/200
800/800 [==============================] - 0s 59us/step - loss: 1.3125 - val_loss: 1.3866
Epoch 155/200
800/800 [==============================] - 0s 60us/step - loss: 1.6083 - val_loss: 1.2214
Epoch 156/200
800/800 [==============================] - 0s 60us/step - loss: 1.1929 - val_loss: 1.1856
Epoch 157/200
800/800 [==============================] - 0s 65us/step - loss: 1.2757 - val_loss: 1.2999
Epoch 158/200
800/800 [==============================] - 0s 59us/step - loss: 1.1631 - val_loss: 1.3089
Epoch 159/200
800/800 [==============================] - 0s 66us/step - loss: 1.2421 - val_loss: 1.1397
Epoch 160/200
800/800 [==============================] - 0s 62us/step - loss: 1.1114 - val_loss: 1.1602
Epoch 161/200
800/800 [==============================] - 0s 64us/step - loss: 

## Price optimization

In [16]:
# make model copy for price
model_p = keras.models.clone_model(model)
model_p.set_weights(model.get_weights())

In [17]:
# "freeze" the model
# we want our model for demand to stay the same
# while the price input is optimized
for layer in model_p.layers:
    layer.trainable = False

In [18]:
# dict of layers
ls = {l.name:l for l in model_p.layers}
ls

{'concat': <keras.layers.merge.Concatenate at 0x1259eeba8>,
 'hid': <keras.layers.core.Dense at 0x125903780>,
 'inp_p': <keras.engine.topology.InputLayer at 0x11c87f978>,
 'inp_x': <keras.engine.topology.InputLayer at 0x11c87fcc0>,
 'out': <keras.layers.core.Dense at 0x1259f1780>}

In [19]:
# add a single unit dense inbetween price input and hidden layer
# this will create a single trainable weight
# we will feed the input all 1's so it can learn the optimal price
p = Dense(1, use_bias=False, name='price')(ls['inp_p'].output)

In [20]:
# concat the inputs and send through hidden layers like before
x = ls['concat']([p, ls['inp_x'].output])
x = ls['hid'](x)
x = ls['out'](x)

In [21]:
# our output of the last model was an estimate for demand
# so now we multiply that by the trained price to get revenue
out = Multiply(name='revenue_out')([p, x])

In [22]:
# build model
model_p = Model([ls['inp_p'].input, ls['inp_x'].input], out)

In [23]:
# we want to maximize revenue, but keras minizises loss
# so we'll just make our loss the negative revenue
def rev_loss(y_true, y_pred):
    return -K.mean(y_pred)

In [24]:
# compile model
model_p.compile(Adam(lr=.1), loss=rev_loss)

In [25]:
# callback to track layer weights
class WtTracker(keras.callbacks.Callback):
    def __init__(self, layer, display=10):
        self.seen = 0
        self.layer = layer
        self.display = display

    def on_epoch_end(self,batch,logs={}):
        self.seen += 1
        if self.seen % self.display == 0:
            wts = self.layer.get_weights()[0].flatten().round(2)
            print(f'epoch {self.seen}, price {wts}')

In [27]:
# train
hist = model_p.fit([np.ones((len(xs), 1)), xs], y,
                   epochs=200,
                   verbose=0,
                   callbacks=[WtTracker(model_p.get_layer('price'))])

epoch 10, price [29.89]
epoch 20, price [54.09]
epoch 30, price [72.58]
epoch 40, price [85.48]
epoch 50, price [93.42]
epoch 60, price [97.6]
epoch 70, price [99.43]
epoch 80, price [100.08]
epoch 90, price [100.26]
epoch 100, price [100.3]
epoch 110, price [100.3]
epoch 120, price [100.3]
epoch 130, price [100.3]
epoch 140, price [100.3]
epoch 150, price [100.3]
epoch 160, price [100.3]
epoch 170, price [100.3]
epoch 180, price [100.3]
epoch 190, price [100.3]
epoch 200, price [100.31]
